# Part 8 - Introdução a Planos


### Context 

We introduce here an object which is crucial to scale to industrial Federated Learning: the Plan. It reduces dramatically the bandwidth usage, allows asynchronous schemes and give more autonomy to remote devices. The original concept of plan can be found in the paper [Towards Federated Learning at Scale: System Design](https://arxiv.org/pdf/1902.01046.pdf), but it has been adapted to our needs in the PySyft library.

A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. This way, to compute remotely this sequence of $n$ operations on some remote input referenced through pointers, instead of sending $n$ messages you need now to send a single message with the references of the plan and the pointers. You can also provide tensors with your function (that we call _state tensors_) to have extended functionalities. Plans can be seen either like a function that you can send, or like a class which can also be sent and executed remotely. Hence, for high level users, the notion of plan disappears and is replaced by a magic feature which allow to send to remote workers arbitrary functions containing sequential torch functions.

One thing to notice is that the class of functions that you can transform into plans is currently limited to sequences of hooked torch operations exclusively. This excludes in particular logical structures like `if`, `for` and `while` statements, even if we are working to have workarounds soon. _To be completely precise, you can use these but the logical path you take (first `if` to False and 5 loops in `for` for example) in the first computation of your plan will be the one kept for all the next computations, which we want to avoid in the majority of cases._

Autores:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)
- Bobby Wagner - Twitter [@bobbyawagner](https://twitter.com/bobbyawagner) - GitHub: [@robert-wagner](https://github.com/robert-wagner)
- Marianne Monteiro - Twitter [@hereismari](https://twitter.com/hereismari) - GitHub: [@mari-linhares](https://github.com/mari-linhares)	

Tradutor:
- João Lucas - GitHub: [@joaolcaas](https://github.com/joaolcaas)

### Importações e especificações do modelo

Primeiro faremos as importações oficiais.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

E então aqueles específicos do PySyft, com uma importante observação: **o worker local não deve ser um worker cliente**. Workers não clientes podem salvar objetos e precisamos dessa habilidade para usar um plano.

In [3]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

Para ser consistente com o conceito fornecido no artigo referenciado, definimos _workers_ remotos ou _devices_, provendo eles com algum dado. 

In [4]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

### Exemplo Básico

Vamos definir uma função que queremos transformar em um plano. Para isso, apenas adiciona-se um _decorator_ acima da definição da função.

In [5]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

Vamos verificar.
Sim, agora nós temos um plano!

In [6]:
plan_double_abs

<Plan plan_double_abs id:89796211423 owner:me>

Para usar um plano, precisamos de duas coisas: contruir um plano _(i.e. registrar uma sequência de operações presentes na função)_ e mandar isso para um worker / device. Felizmente você pode fazer isso de uma maneira muito fácil.

#### Construindo um plano

Para construir um plano, você só precisa chamar isso em algum dado.

Vamos primeiro pegar a referência para algum dado remoto: uma requisição é enviada para a rede e o ponteiro de referência é retornado.

In [7]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

(Wrapper)>[PointerTensor | me:33127708416 -> device_1:47766305516]
	Tags: input_data 
	Shape: torch.Size([2])

Se dissermos para um plano que ele deve ser executado remotamente no dispositivo `location:device_1`, isso retornará um erro pois o plano não foi construído ainda.

In [8]:
plan_double_abs.is_built

False

In [9]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


Para construir um plano só precisamos chamar `build` no plano e passar os argumentos necessários para a execução de um plano (a.k.a algum dado). Quando um plano é construído, todos os comandos são executados sequencialmente por um worker local, capturados pelo plano e então salvos no atributo `readable_plan`!

In [10]:
plan_double_abs.build(torch.tensor([1., -2.]))

In [11]:
plan_double_abs.is_built

True

Agora, se tentarmos enviar um plano, irá funcionar!

In [12]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:85123354948 -> device_1:89796211423]

Como nos tensores, podemos ver um ponteiro para o objeto que foi enviado, sendo chamado simplesmente de `PointerPlan`.

Um lembrete importante é que, quando um plano é construído, nós de antemão pré-definimos os id(s) dos resultados das computações que devem ser salvos. Isso irá permitir enviar os comandos de forma assíncrona, já ter uma referência para de um resultado virtual e continuar as computações locais sem ter que esperar os resultados remotos serem computados. Uma aplicação importante é quando você exige a computação de um grupo no device_1 e não quer esperar pelo fim dessa computação para mandar outro grupo de computação para o device_2

#### Executando um Plano Remotamente

Nós agora podemos executar um plano remotamente chamando um ponteiro para um plano com um ponteiro para algum dado. Isso emite um comando para executar esse plano remotamente, sendo assim, a localização pré-definida da saída do plano agora contém o resultado (lembre-se que  pré-definimos a localização do resultado antes da computação). Isso também requer uma única rodada de comunicação.

O resultado é simplesmente um ponteiro, assim como quando você chamada uma função do Torch enganchada (hooked).

In [ ]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

E você pode simplesmentes pegar o valor de volta.

In [ ]:
pointer_to_result.get()

### Towards a concrete example

Mas nós queremos que Planos sejam aplicados para aprendizado profundo e federado, certo? Então vamos dar uma olhada em um exemplo ligeiramente mais complicado, usando rede neural que certamente você deve estar ansioso para usar.
Note que estamos agora transformando uma classe em um Plano. Para isso, nós herdamos a nossa classe de sy.Plan (ao invés de herdar de nn.Module)

In [13]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)


In [14]:
net = Net()

In [15]:
net

<Net Net id:58498301328 owner:me>

Vamos construir um plano usando dados de exemplo.

In [16]:
net.build(torch.tensor([1., 2.]))

Agora enviamos o Plano para um worker remoto.

In [18]:
pointer_to_net = net.send(device_1)
pointer_to_net

[PointerPlan | me:66386943511 -> device_1:58498301328]

Vamos recuperar alguns dados remotos.

In [19]:
pointer_to_data = device_1.search('input_data')[0]

A sintaxe é como a execução sequencial remota normal, isso é, como uma execução local. Mas comparada a uma execução remota clássica, tem apenas uma rodada de comunicação para cada execução.

In [20]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:87899785368 -> device_1:93434720381]

E recebemos o resultado como sempre!

In [ ]:
pointer_to_result.get()

Et voilà! Nós vimos como dramaticamente reduzir a comunicação entre worker local (ou servidor) e os dispositivos remotos.

### Switch between workers

One major feature that we want to have is to use the same plan for several workers, that we would change depending on the remote batch of data we are considering.
In particular, we don't want to rebuild the plan each time we change of worker. Let's see how we do this, using the previous example with our small network.

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [ ]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

Here are the main steps we just executed

In [ ]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

And actually you can build other PointerPlans from the same plan, so the syntax is the same to run remotely a plan on another device

In [ ]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

> Note: Currently, with Plan classes, you can only use a single method and you have to name it "forward".

### Automatically building plans that are functions

For functions (`@` `sy.func2plan`) we can automatically build the plan with no need to explicitly calling `build`, actually in the moment of creation the plan is already built.

To get this functionality the only thing you need to change when creating a plan is setting an argument to the decorator called `args_shape` which should be a list containing the shapes of each argument.

In [ ]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

O parâmetro `args_shape` é usado internamente para criar tensores de exemplo com o formato em qual foi passado para construir o plano.

In [ ]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

Também é possível prover
You can also provide state elements to functions!

In [ ]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [ ]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

Para aprender mais sobre isso, você pode descobrir como usar Planos com Protocolos no tutorial Part 8 bis!

### Dê-nos uma estrela em nosso repo do PySyft no GitHub

A maneira mais fácil de ajudar nossa comunidade é adicionando uma estrela nos nossos repositórios! Isso ajuda a aumentar a conscientização sobre essas ferramentas legais que estamos construindo.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Veja nossos tutoriais no GitHub!

Fizemos tutoriais muito bons para entender melhor como deve ser a Aprendizagem Federada e a proteção de Privacidade, e como estamos construindo as coisas básicas que precisamos para fazer com que isso aconteça.

- [Tutoriais do PySyft](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)

### Junte-se ao Slack!

A melhor maneira de manter-se atualizado sobre os últimos avanços é se juntar à nossa comunidade! 

- [http://slack.openmined.org](http://slack.openmined.org)

### Contribua com o projeto!

A melhor maneira de contribuir para a nossa comunidade é se tornando um contribuidor do código! A qualquer momento, você pode acessar a página de *Issues* (problemas) do PySyft no GitHub e filtrar por "Projetos". Isso mostrará todas as etiquetas (tags) na parte superior, com uma visão geral de quais projetos você pode participar! Se você não deseja ingressar em um projeto, mas gostaria de codificar um pouco, também pode procurar mais mini-projetos "independentes" pesquisando problemas no GitHub marcados como "good first issue".

- [Etiquetados como Good First Issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Doar

Se você não tem tempo para contribuir com nossa base de códigos, mas ainda deseja nos apoiar, também pode se tornar um Apoiador em nosso Open Collective. Todas as doações vão para hospedagem na web e outras despesas da comunidade, como hackathons e meetups!

[Página do Open Collective do OpenMined](https://opencollective.com/openmined)